In [14]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from pprint import pprint

load_dotenv("../.env")

client = OpenAI(api_key=os.environ['LLM_API_KEY'])

MODEL_NAME = "gpt-4o-mini"

In [15]:
import json

def get_search_result(keyword, when="1", unit="m"):
    result = {
        "result": [
            {"title": "1914年（大正3年）中央停車場改め「東京駅」として営業を開始しました"},
            {"title": "1929年（昭和4年）八重洲口が開設されました"},
            {"title": "2014年（平成26年）東京駅開業100周年を迎えました"},
        ]
    }

    return json.dumps(result)


In [16]:
from openai.types.chat import ChatCompletionToolParam

tools=[
    ChatCompletionToolParam({
        "type": "function",
        "function": {
            "name": "get_search_result",
            "description": "指定したキーワードの検索結果を取得する",
            "parameters": {
                "type": "object",
                "properties": {
                    "keyword": {
                        "type": "string",
                        "description": "キーワード",
                    },
                    "when": {
                        "type": "number",
                        "description": "日付や時間の範囲"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["d", "h", "y"]
                    },
                },
                "required": ["keyword"],
            },
        },
    })
]


In [17]:
question = "日本の首都はどこにありますか？"

response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": question},
    ],
    tools=tools,
    tool_choice="auto",
)

pprint(vars(response))

{'_request_id': 'req_e4531c2891214e929729c7cb052012bd',
 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='日本の首都は東京にあります。', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, annotations=[]))],
 'created': 1771998754,
 'id': 'chatcmpl-DD270K9LIvF2MYNR0rj3Kcp4pDOvT',
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'chat.completion',
 'service_tier': 'default',
 'system_fingerprint': 'fp_bfa3527827',
 'usage': CompletionUsage(completion_tokens=10, prompt_tokens=91, total_tokens=101, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))}


In [18]:
question = "東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください"

response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": question},
    ],
    tools=tools,
    tool_choice="auto",
)

# レスポンスの表示
pprint(vars(response))


{'_request_id': 'req_89484063eab64144af92c23b17a64590',
 'choices': [Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_iYzFZHsNKbDQ35mgQ95ftXD6', function=Function(arguments='{"keyword":"東京駅 イベント","when":30,"unit":"d"}', name='get_search_result'), type='function')], annotations=[]))],
 'created': 1771998755,
 'id': 'chatcmpl-DD271SU4IY7QK9sROa5A7eWiIJKqC',
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'chat.completion',
 'service_tier': 'default',
 'system_fingerprint': 'fp_bfa3527827',
 'usage': CompletionUsage(completion_tokens=26, prompt_tokens=97, total_tokens=123, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))}


In [19]:
if response.choices[0].finish_reason == 'tool_calls':
    tool = response.choices[0].message.tool_calls[0]

    function_name = tool.function.name
    print(f"関数名：{function_name}")

    arguments = json.loads(tool.function.arguments)
    print(f"引数：{arguments}")

    function_response = globals()[function_name](**arguments)
    print("関数の実行結果：")
    pprint(json.loads(function_response))

    response_after_tool_call = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "user", "content": question},
            response.choices[0].message,
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": function_response,
            },
        ],
    )

    print("言語モデルからの回答：")
    print(response_after_tool_call.choices[0].message.content.strip())

else:
    print("言語モデルからの回答：")
    print(response.choices[0].message.content.strip())


関数名：get_search_result
引数：{'keyword': '東京駅 イベント', 'when': 30, 'unit': 'd'}
関数の実行結果：
{'result': [{'title': '1914年（大正3年）中央停車場改め「東京駅」として営業を開始しました'},
            {'title': '1929年（昭和4年）八重洲口が開設されました'},
            {'title': '2014年（平成26年）東京駅開業100周年を迎えました'}]}
言語モデルからの回答：
最近1ヶ月以内の東京駅のイベントについて、以下の情報があります：

1. **1914年（大正3年）中止駅場改名「東京駅」として営業を開始しました**
2. **1929年（昭和4年）八重洲口が開設されました**
3. **2014年（平成26年）東京駅開業100周年を迎えました**

これらのイベントは主に歴史的な記念行事や改装に関するものであり、現在も東京駅に関連する特別な催しや展示が行われている可能性があります。具体的なイベントの詳細については、東京駅の公式サイトや関連の情報発信をチェックすると良いでしょう。
